#### 1.Bibliothèques 

In [52]:
import numpy as np
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, r2_score

#### 2. Chargement de données des attributs pertinents (4/7 seulement) depuis le fichier 'insurance.csv'

In [51]:
data = pd.read_csv('insurance.csv')
data.columns
data_sample = data[['age','bmi', 'children', 'charges']]
print (data_sample.shape)
print (data_sample.columns)

(1338, 4)
Index(['age', 'bmi', 'children', 'charges'], dtype='object')


#### 3.  Correction de données manquantes : exemple de complétion de ces valeurs par la moyenne de la colonne
PS : Nous n'avons pas de valeurs manquantes donc ce bout de code ne va pas influencer notre code ou données

In [50]:
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data_sample)
data=imp.transform(data_sample) 

#### 4. Séprataion des attributs (Inputs) et de l'attribut objectif (Output)

In [39]:
NB_FEATURES=3
X=data[:,0:NB_FEATURES]         # Features: inputs (3 premières colonnes)
y=data[:,NB_FEATURES:].ravel()  # Target : output/ (Dernière colonne)

#### 5. Découpage de l'ensemble de données : 80% TRAINING, 20% TEST

In [53]:
TEST_SIZE = 0.20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,shuffle=True)

#### 6. Affihage des quelques informations concernant notre dataset 

In [54]:
print(f" >>> Taille du dataset / nombre d'instances :\t {data.shape[0]} ")
print(f" >>> Nombre d'attributs / Inputs : \t\t {NB_FEATURES} ")
print(f" >>> Taille du dataset d'entrainement : \t {X_train.shape[0]}")
print(f" >>> Taille du dataset de test : \t\t {X_test.shape[0]}")

 >>> Taille du dataset / nombre d'instances :	 1338 
 >>> Nombre d'attributs / Inputs : 		 3 
 >>> Taille du dataset d'entrainement : 	 1070
 >>> Taille du dataset de test : 		 268


#### 7. Normalisation des données du dataset

In [42]:
scaler=preprocessing.StandardScaler().fit(X_train)
X_train=scaler.transform(X_train)
X_test=scaler.transform(X_test)

# Etape 2 : Paramètres par défaut
#### 8. Création d'un modèle de regression avec les paramètres par défaut de ScikitLearn + apprentissage du modèle

In [55]:
regr = MLPRegressor(hidden_layer_sizes=(100,), activation='relu',  solver='adam',
                   alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001,
                   power_t=0.5, max_iter=200, shuffle=True, random_state=1, tol=0.0001, verbose=False,
                   warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False,
                   validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, 
                   )

regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

C:\Users\HP\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:566: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


#### 9. Métriques de regression

In [57]:
RSS=mean_squared_error(y_test, y_pred)*89
print(" RSS = %.2f" % RSS)
print(" RSE = %.2f" % (1-r2_score(y_test, y_pred)))
print(" R2 (Coefficient de determination) = %.2f" % r2_score(y_test, y_pred))

 RSS = 11310517364.96
 RSE = 0.97
 R2 (Coefficient de determination) = 0.03


# Etape 3 : GridSearchCV
#### 10. Utilisation de la méthode GridSearchCV qui trouve facilement les valeurs optimales parmi les valeurs données pour les paramètres pertinents (activation, solver, learning_rate, max_iter)

In [ ]:
from sklearn.model_selection import GridSearchCV

mlp_gs = MLPRegressor(max_iter=3000)
parameter_space = {
    'activation': ['relu', 'identity', 'logistic', 'tanh'],
    'solver': ['adam', 'lbfgs', 'sgd'],
    'learning_rate': ['constant','adaptive', 'invscaling'],
    'max_iter': [200, 2000, 3000], 

}
#n_jobs=-1 , -1 sert à utiliser tous les cœurs de processeur disponibles.
#cv = 10 est pour la validation croisée, ici cela signifie 10 fois la validation croisée stratifiée (K-fold)
clf = GridSearchCV(mlp_gs, parameter_space, n_jobs=-1, cv=10)

#Apprentissage / Entrainement de modèle
clf.fit(X, y)

#Affichage des scores pour toutes les combinaisons (modèles) possibles 
means = clf.cv_results_['mean_test_score']
stds = clf.cv_results_['std_test_score']

i=0
for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    i=i+1    
print("Le nombre de combinaisons (modèles) possible est : %r " % (i)) 
print(" \n ")
print("|  Mean  |   Std   |  Combinaison / Modèle ")
print("-------------------------------------------------------------------------------------------------------------------")

for mean, std, params in zip(means, stds, clf.cv_results_['params']):
    print("%0.3f    | %0.03f   | %r" % (mean, std, params))

#### 11. Affichage du meilleur modèle (hyperparamètres optimaux parmi les valeurs données). 

In [ ]:
print('Meilleurs paramètres trouvés : \n', clf.best_params_)

#### 12. Test du meilleur modèle sur l'ensemble de test.

In [ ]:
y_test, y_pred = y_test , clf.predict(X_test)

#### 13. Métriques de regression

In [ ]:
RSS=mean_squared_error(diabetes_y_test, diabetes_y_pred)*89
print(" RSS = %.2f" % RSS)
print(" RSE = %.2f" % (1-r2_score(diabetes_y_test, diabetes_y_pred)))
print(" R2 (Coefficient de determination) = %.2f" % r2_score(diabetes_y_test, diabetes_y_pred))